In [1]:
# Testing Notebook
# Final code in qkd.py

In [ ]:
# BB84

# Basis = {"rectilinear", "diagonal"}
# Polarization = {"horizontal", "vertical", "pos_diag", "neg_diag"}

basis = ["rectilinear", "diagonal"]

polarization = {"rectilinear": ["horizontal", "vertical"],
                "diagonal": ["pos_diag", "neg_diag"]}


In [ ]:
# qubit

class qubit():
    
    self.__init__